**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install scikit-video

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
from random import sample
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Flatten, Dropout
from keras.optimizers import sgd, adam
from keras.layers import (
    Conv2D,
    MaxPooling2D,
    Activation,
    AveragePooling2D,
    Reshape,
    BatchNormalization,
)

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass

    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass

    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action

    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else:  # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` allows the agent to choose an action given its current state. Here, it follows an $\varepsilon$-greedy exploration: it takes a random action with probability $\varepsilon \in [0, 1]$ among the set of all possible actions. Otherwise, it chooses the **greedy** action $a^* \in \arg \max_a Q^\pi (s, a)$ (where $s$ is the agent's current state).

$\varepsilon$ is essential as it allows the agent to sufficiently explore the possible states and update their $Q$-values. The value of $\varepsilon$ is usually decayed during the training to force the agent to explore more at the beggining.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros(
            (max_time + 2, grid_size * self.scale, grid_size * self.scale, 3)
        )

    def draw(self, e):
        skvideo.io.vwrite(str(e) + ".mp4", self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0

        b = cv2.resize(
            b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST
        )

        self.to_draw[t, :, :, :] = b

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError("Error: action not recognized")

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )
        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros(
            (
                self.max_time + 2,
                self.grid_size * self.scale,
                self.grid_size * self.scale,
                3,
            )
        )

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )

        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50  # set small when debugging
epochs_test = 10  # set small when debugging
beta = 5.  # epsilon decay speed

# display videos
def display_videos(name):
    video = io.open(name, "r+b").read()
    encoded = base64.b64encode(video)
    return """<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>""".format(
        encoded.decode("ascii")
    )

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`position` is an array that keeps track of the player position (1 where the player is, -1 on edges, and 0 elsewhere). `board` gives the rewards of each position (0.5 for a cheese, -1 for poison and 0 elsewhere). 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Take a random action (between the 4 possible actions)
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=""):
    # Number of won games
    score = 0

    for e in range(1, epochs + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print(
            "Win/lose count {}/{}. Average score ({})".format(
                win, lose, score / (1 + e)
            )
        )
    print("Final score: " + str(score / epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix="random")
HTML(display_videos("random10.mp4"))

Win/lose count 6.5/13.0. Average score (-3.25)
Win/lose count 8.0/17.0. Average score (-5.166666666666667)
Win/lose count 6.0/13.0. Average score (-5.625)
Win/lose count 12.0/12.0. Average score (-4.5)
Win/lose count 7.5/19.0. Average score (-5.666666666666667)
Win/lose count 12.5/12.0. Average score (-4.785714285714286)
Win/lose count 12.0/18.0. Average score (-4.9375)
Win/lose count 8.5/7.0. Average score (-4.222222222222222)
Win/lose count 11.0/17.0. Average score (-4.4)
Win/lose count 7.0/11.0. Average score (-4.363636363636363)
Final score: -4.8


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
We have by definition:
\begin{align*}
    Q^\pi (s, a) &= E_{p^\pi}\left[\left.\sum_{t=0}^\infty \gamma^tr(s_t, a_t)\right|s_0=s, a_0 = a \right]\\
    &=r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}\left[E_{p^\pi}\left[\left.\sum_{t=1}^\infty \gamma^{t-1}r(s_t, a_t)\right|s_1=s', a_1 = a' \right]\right]\\
    &=r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}\left[E_{p^\pi}\left[\left.\sum_{t=0}^\infty \gamma^{t}r(s_{t+1}, a_{t+1})\right|s_1=s', a_1 = a' \right]\right]
\end{align*}

But, by homogeneity:
\begin{align*}
    E_{p^\pi}\left[\left.\sum_{t=0}^\infty \gamma^{t}r(s_{t+1}, a_{t+1})\right|s_1=s', a_1 = a' \right] &= E_{p^\pi}\left[\left.\sum_{t=0}^\infty \gamma^tr(s_t, a_t)\right|s_0=s', a_0 = a' \right]\\
    &=Q^\pi(s', a')
\end{align*}

Then, we finally have:
\begin{align*}
    Q^\pi (s, a) &= r(s, a) + \gamma E_{(s',a')\sim p(.|s,a)}\left[Q^\pi(s', a')\right]\\
    &= E_{(s',a')\sim p(.|s,a)}\left[r(s, a) + \gamma Q^\pi(s', a')\right]
\end{align*}

***
We can assume that the optimal policy is deterministic and denote by $\pi^*(s)$ the best action from the state $s$. By the first equality, we have:
\begin{align*}
Q^{*}(s,a) &= E_{(s', a')\sim p^{\pi^*}(.|s,a)}\left[r(s,a)+\gamma Q^{*}(s',a')\right]\\
&= E_{s'\sim \pi^{*}(.|s,a)}\left[r(s,a)+\gamma Q^{*}(s',\pi^*(s'))\right]\\
&= E_{s'\sim \pi^{*}(.|s,a)}\left[r(s,a)+\gamma \max_{a'}Q^{*}(s',a')\right]
\end{align*}
***

We want to minimize the TD error at each time $t$ given by $\delta_t(\theta) = r_t + \gamma \max_{a'}Q(s_
{t+1}, a', \theta) - Q(s_t, a_t, \theta)$ . Then the objective:
\begin{equation*}
    \mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\left[\left\| r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\right\|^{2}\right]
\end{equation*}
is plausible.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.position = 0

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[self.position % self.max_memory] = m

        self.position += 1

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent, env, epoch, prefix=""):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(
            "Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})".format(
                e, epoch, loss, win, lose, win - lose
            )
        )
        agent.save(name_weights=prefix + "model.h5", name_model=prefix + "model.json")

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(
        self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2
    ):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99

        self.grid_size = grid_size

        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)

        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape((1, 5, 5, self.n_state)))
        return q.argmax(-1)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])

        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape((1, 5, 5, self.n_state)))

            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.model.predict(
                    s_.reshape((1, 5, 5, self.n_state))
                ).max(-1)
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights="model.h5", name_model="model.json"):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)

    def load(self, name_weights="model.h5", name_model="model.json"):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model


class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)

        # NN Model

        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(64, activation="relu"))
        model.add(Dropout(0.2))
        model.add(Dense(self.n_action, activation="linear"))

        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix="fc_train")
HTML(display_videos("fc_train{}.mp4".format((epochs_train // 10) * 10)))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 001/050 | Loss 0.0071 | Win/lose count 4.0/4.0 (0.0)
Epoch 002/050 | Loss 0.0184 | Win/lose count 5.5/3.0 (2.5)
Epoch 003/050 | Loss 0.0035 | Win/lose count 2.5/1.0 (1.5)
Epoch 004/050 | Loss 0.0116 | Win/lose count 3.0/2.0 (1.0)
Epoch 005/050 | Loss 0.0039 | Win/lose count 0/2.0 (-2.0)
Epoch 006/050 | Loss 0.0017 | Win/lose count 3.5/4.0 (-0.5)
Epoch 007/050 | Loss 0.0083 | Win/lose count 5.0/2.0 (3.0)
Epoch 008/050 | Loss 0.0029 | Win/lose count 5.0/1.0 (4.0)
Epoch 009/050 | Loss 0.0375 | Win/lose count 3.5/1.0 (2.5)
Epoch 010/050 | Loss 0.0158 | Win/lose count 4.0/4.0 (0.0)
Epoch 011/050 | Loss 0.0062 | Win/lose count 1.0/1.0 (0.0)
Epoch 012/050 | Loss 0.0141 | Win/lose count 4.0/6.0 (-2.0)
Epoch 013/050 | Loss 0.0048 | Win/lose count 5.0/6.0 (-1.0)
Epoch 014/050 | Loss 0.0104 | Win/lose count 6.0/5.0 (1.0)
Epoch 015/050 | Loss 0.0240 | Win/lose count 2.5/5.0 (-2

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)

        model = Sequential()
        model.add(
            Conv2D(
                input_shape=(5, 5, self.n_state),
                filters=16,
                kernel_size=2,
                activation="relu",
            )
        )
        model.add(Conv2D(filters=32, kernel_size=2, activation="relu"))
        model.add(Flatten())
        model.add(Dense(self.n_action, activation="linear"))

        model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.001, epsilon=0.1, memory_size=2000, batch_size=32)
train(agent, env, epochs_train, prefix="cnn_train")
HTML(display_videos("cnn_train{}.mp4".format((epochs_train // 10) * 10)))

Epoch 001/050 | Loss 0.0001 | Win/lose count 3.5/2.0 (1.5)
Epoch 002/050 | Loss 0.0006 | Win/lose count 2.5/3.0 (-0.5)
Epoch 003/050 | Loss 0.0016 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/050 | Loss 0.0039 | Win/lose count 8.0/5.0 (3.0)
Epoch 005/050 | Loss 0.0015 | Win/lose count 2.5/2.0 (0.5)
Epoch 006/050 | Loss 0.0065 | Win/lose count 4.0/1.0 (3.0)
Epoch 007/050 | Loss 0.0514 | Win/lose count 2.5/4.0 (-1.5)
Epoch 008/050 | Loss 0.0009 | Win/lose count 6.0/3.0 (3.0)
Epoch 009/050 | Loss 0.0205 | Win/lose count 6.5/3.0 (3.5)
Epoch 010/050 | Loss 0.0032 | Win/lose count 3.5/3.0 (0.5)
Epoch 011/050 | Loss 0.0018 | Win/lose count 8.0/4.0 (4.0)
Epoch 012/050 | Loss 0.1163 | Win/lose count 6.0/1.0 (5.0)
Epoch 013/050 | Loss 0.0052 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/050 | Loss 0.0006 | Win/lose count 8.0/2.0 (6.0)
Epoch 015/050 | Loss 0.0022 | Win/lose count 4.5/3.0 (1.5)
Epoch 016/050 | Loss 0.0006 | Win/lose count 9.0/4.0 (5.0)
Epoch 017/050 | Loss 0.0005 | Win/lose count 5.0/3.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 9.5/3.0. Average score (3.25)
Win/lose count 3.0/2.0. Average score (2.5)
Win/lose count 12.0/2.0. Average score (4.375)
Win/lose count 4.5/0. Average score (4.4)
Win/lose count 3.5/1.0. Average score (4.083333333333333)
Win/lose count 8.0/9.0. Average score (3.357142857142857)
Win/lose count 7.0/0. Average score (3.8125)
Win/lose count 6.0/3.0. Average score (3.7222222222222223)
Win/lose count 2.5/3.0. Average score (3.3)
Win/lose count 6.0/7.0. Average score (2.909090909090909)
Final score: 3.2
Test of the FC
Win/lose count 2.5/5.0. Average score (-1.25)
Win/lose count 5.0/6.0. Average score (-1.1666666666666667)
Win/lose count 5.5/7.0. Average score (-1.25)
Win/lose count 5.0/6.0. Average score (-1.2)
Win/lose count 4.0/4.0. Average score (-1.0)
Win/lose count 1.5/1.0. Average score (-0.7857142857142857)
Win/lose count 3.0/7.0. Average score (-1.1875)
Win/lose count 5.5/10.0. Average score (-1.5555555555555556)
Win/lose count 4.0/2.0. Average score (-1

In [0]:
HTML(display_videos('cnn_test{}.mp4'.format((epochs_test // 10) * 10)))

In [0]:
HTML(display_videos('fc_test{}.mp4'.format((epochs_test // 10) * 10)))

We can observe that the convolution model has better performances than the fully connected model. However, each of those performances are far from being optimal. As we can see, the agents are often stuck in local neighborhood doing cycles and do not sufficiently explore.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix='', eps_min=0.1, eps_max=1.0):
    # Number of won games
    score = 0
    loss = 0
    total_steps = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

            # Update epsilon (exp decay)
            epsilon = eps_min + (eps_max - eps_min) * np.exp(- beta * total_steps / (epoch * env.max_time))
            agent.set_epsilon(epsilon)
            total_steps += 1
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(
            "Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.3f}/{:.3f} ({:.3f}) | eps {:.4f}".format(
                e, epoch, loss, win, lose, win - lose, agent.epsilon
            )
        )
        agent.save(name_weights=prefix + "model.h5", name_model=prefix + "model.json")
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        self.malus_position = np.zeros((grid_size, grid_size))
      
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError("Error: action not recognized")

        self.t = self.t + 1
        reward = 0.0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (
                self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )
        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros(
            (
                self.max_time + 2,
                self.grid_size * self.scale,
                self.grid_size * self.scale,
                3,
            )
        )

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate(
            (
                self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )

        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore', eps_min=0.1, eps_max=0.9)
HTML(display_videos('cnn_train_explore{}.mp4'.format((epochs_train // 10) * 10)))

Epoch 001/050 | Loss 0.0103 | Win/lose count 15.500/24.100 (-8.600) | eps 0.8239
Epoch 002/050 | Loss 0.0062 | Win/lose count 10.000/26.500 (-16.500) | eps 0.7547
Epoch 003/050 | Loss 0.0080 | Win/lose count 8.000/24.900 (-16.900) | eps 0.6921
Epoch 004/050 | Loss 0.0049 | Win/lose count 10.500/25.600 (-15.100) | eps 0.6355
Epoch 005/050 | Loss 0.0027 | Win/lose count 7.500/20.500 (-13.000) | eps 0.5843
Epoch 006/050 | Loss 0.0095 | Win/lose count 15.500/26.400 (-10.900) | eps 0.5380
Epoch 007/050 | Loss 0.0043 | Win/lose count 4.500/25.400 (-20.900) | eps 0.4961
Epoch 008/050 | Loss 0.0072 | Win/lose count 17.000/24.100 (-7.100) | eps 0.4582
Epoch 009/050 | Loss 0.0028 | Win/lose count 15.000/24.200 (-9.200) | eps 0.4240
Epoch 010/050 | Loss 0.0224 | Win/lose count 13.500/21.500 (-8.000) | eps 0.3930
Epoch 011/050 | Loss 0.0125 | Win/lose count 15.500/20.700 (-5.200) | eps 0.3650
Epoch 012/050 | Loss 0.0066 | Win/lose count 14.000/18.700 (-4.700) | eps 0.3396
Epoch 013/050 | Loss 0.01

In [0]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore{}.mp4'.format((epochs_test // 10) * 10)))

Win/lose count 10.5/6.0. Average score (2.25)
Win/lose count 23.0/5.0. Average score (7.5)
Win/lose count 13.0/3.0. Average score (8.125)
Win/lose count 21.0/4.0. Average score (9.9)
Win/lose count 21.5/4.0. Average score (11.166666666666666)
Win/lose count 18.0/4.0. Average score (11.571428571428571)
Win/lose count 10.5/2.0. Average score (11.1875)
Win/lose count 30.0/4.0. Average score (12.833333333333334)
Win/lose count 13.0/2.0. Average score (12.65)
Win/lose count 25.5/6.0. Average score (13.272727272727273)
Final score: 14.6


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.